dataset link https://drive.google.com/file/u/2/d/1hEL7PFv7_TRjmdWSp4l9_bh8FTA5fl5p/view?usp=sharing

In [ ]:
# import library
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_excel('/content/drive/My Drive/DATASET-LATIHAN/online_retail.xlsx')

Mounted at /content/drive


In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
# cleaning data

#1. menghapus ruang ekstra dalam deskripsi
data['Description'] = data['Description'].str.strip()

#2. menghapus baris tanpa nomor faktur (InvoiceNo)
data.dropna(axis=0, subset= ['InvoiceNo'], inplace=True)
data['InvoiceNo'] = data['InvoiceNo'].astype(str)

#3. menghapus semua transaksi yang dilakukan secara kredit
data = data[~data['InvoiceNo'].str.contains('C')]

In [ ]:
# memisahkan data sesuai dengan wilayah transaksi
# prancis
basket_france = (data[data['Country'] == 'France'].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

# portugal
basket_portugal = (data[data['Country'] == 'Portugal'].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

# swedia
basket_swedia = (data[data['Country'] == 'Sweden'].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
pd.set_option('display.max_columns', None)

In [ ]:
basket_swedia

Description,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE WOODLAND,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 RAFFIA RIBBONS VINTAGE CHRISTMAS,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,4 PEAR BOTANICAL DINNER CANDLES,4 VANILLA BOTANICAL CANDLES,5 HOOK HANGER MAGIC TOADSTOOL,6 GIFT TAGS 50'S CHRISTMAS,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,60 CAKE CASES VINTAGE CHRISTMAS,60 TEATIME FAIRY CAKE CASES,72 SWEETHEART FAIRY CAKE CASES,ABC TREASURE BOOK BOX,ADVENT CALENDAR GINGHAM SACK,AIRLINE BAG VINTAGE TOKYO 78,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOURS SILK FAN,BAG 250g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING MOULD HEART MILK CHOCOLATE,BAKING MOULD HEART WHITE CHOCOLATE,BAKING MOULD ROSE MILK CHOCOLATE,BAKING MOULD ROSE WHITE CHOCOLATE,BAKING MOULD TOFFEE CUP CHOCOLATE,BINGO SET,BLACK KITCHEN SCALES,BLACK LOVE BIRD CANDLE,BLUE HAPPY BIRTHDAY BUNTING,BLUE POLKADOT GARDEN PARASOL,BLUE POLKADOT WRAP,BLUE VINTAGE SPOT BEAKER,BOTTLE BAG RETROSPOT,BOX OF 6 CHRISTMAS CAKE DECORATIONS,BOX OF 6 MINI 50'S CRACKERS,BOX OF 6 MINI VINTAGE CRACKERS,BOYS VINTAGE TIN SEASIDE BUCKET,BUBBLEGUM RING ASSORTED,BUFFALO BILL TREASURE BOOK BOX,BUNDLE OF 3 RETRO NOTE BOOKS,CARAVAN SQUARE TISSUE BOX,CHARLOTTE BAG APPLES DESIGN,CHARLOTTE BAG PINK POLKADOT,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY DOLLY GIRL,CHILDRENS CUTLERY POLKADOT PINK,CHILDRENS TOY COOKING UTENSIL SET,CHRISTMAS METAL POSTCARD WITH BELLS,CINNAMON SCENTED VOTIVE CANDLE,CUPCAKE LACE PAPER SET 6,DINOSAUR PARTY BAG + STICKER SET,DOILEY STORAGE TIN,EAU DE NIL LOVE BIRD CANDLE,FAIRY CAKE FLANNEL ASSORTED COLOUR,FAIRY TALE COTTAGE NIGHTLIGHT,"FEATHER PEN,HOT PINK","FEATHER PEN,LIGHT PINK",FELT FARM ANIMAL WHITE BUNNY,FELT TOADSTOOL SMALL,FELTCRAFT DOLL ROSIE,FELTCRAFT PRINCESS CHARLOTTE DOLL,FIRST AID TIN,GENTLEMAN SHIRT REPAIR KIT,GIFT BAG BIRTHDAY,GIRLS VINTAGE TIN SEASIDE BUCKET,GRAND CHOCOLATECANDLE,GREEN METAL BOX ARMY SUPPLIES,GREEN VINTAGE SPOT BEAKER,GUMBALL COAT RACK,GYMKHANA TREASURE BOOK BOX,HAIRCLIPS FORTIES FABRIC ASSORTED,HAND WARMER BIRD DESIGN,HAND WARMER OWL DESIGN,HAND WARMER UNION JACK,HAPPY STENCIL CRAFT,HEART STRING MEMO HOLDER HANGING,HOMEMADE JAM SCENTED CANDLES,HYACINTH BULB T-LIGHT CANDLES,ICE CREAM BUBBLES,ICE CREAM SUNDAE LIP GLOSS,IVORY LOVE BIRD CANDLE,JAM JAR WITH GREEN LID,JAM JAR WITH PINK LID,JAM MAKING SET PRINTED,JAM MAKING SET WITH JARS,JIGSAW TOADSTOOLS 3 PIECE,JUMBO BAG ALPHABET,JUMBO BAG APPLES,JUMBO BAG OWLS,JUMBO BAG RED RETROSPOT,JUMBO BAG WOODLAND ANIMALS,KIDS RAIN MAC BLUE,KIDS RAIN MAC PINK,KINGS CHOICE TEA CADDY,LARGE PURPLE BABUSHKA NOTEBOOK,LARGE RED BABUSHKA NOTEBOOK,LARGE YELLOW BABUSHKA NOTEBOOK,LIPSTICK PEN BABY PINK,LUNCH BAG APPLE DESIGN,LUNCH BAG CARS BLUE,LUNCH BAG PINK POLKADOT,LUNCH BAG RED RETROSPOT,LUNCH BAG WOODLAND,LUNCH BOX WITH CUTLERY RETROSPOT,MAGIC DRAWING SLATE CIRCUS PARADE,MAGIC DRAWING SLATE DOLLY GIRL,MAGIC DRAWING SLATE PURDEY,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW DOLLY GIRL,MINI JIGSAW PURDEY,MINI JIGSAW SPACEBOY,MINI LIGHTS WOODLAND MUSHROOMS,MINI PAINT SET VINTAGE,MINI PLAYING CARDS BUFFALO BILL,MINI PLAYING CARDS DOLLY GIRL,MINI PLAYING CARDS GYMKHANA,MINI PLAYING CARDS SPACEBOY,MOBILE VINTAGE HEARTS,MONSTERS STENCIL CRAFT,MR ROBOT SOFT TOY,MULTICOLOUR HONEYCOMB PAPER GARLAND,Manual,NOEL GARLAND PAINTED ZINC,ORANGE VOTIVE CANDLE,PACK OF 12 COLOURED PENCILS,PACK OF 12 TRADITIONAL CRAYONS,PACK OF 12 WOODLAND TISSUES,PACK OF 20 NAPKINS PANTRY DESIGN,PACK OF 60 DINOSAUR CAKE CASES,PACK OF 60 PINK PAISLEY CAKE CASES,PACK OF 60 SPACEBOY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,PACK OF 72 SKULL CAKE CASES,PAIR PADDED HANGERS PINK CHECK,PANTRY PASTRY BRUSH,PANTRY SCRUBBING BRUSH,PANTRY WASHING UP BRUSH,PAPER BUNTING PAISLEY PARK,PAPER BUNTING RETROSPOT,PAPER CHAIN KIT RETR

jika dilihat diatas pada invoice 546530
muncul angka 768.0 itu berarti barang dibeli sebanyak 768 kali

- tapi didalam asosiasi kita tidak perlu jumlahnya berapapun, tetap akan dihitungnya 1
- karena akan dibuat boolean muncul / tidak muncul

In [ ]:
basket_swedia.shape

(36, 261)

jadi totalnya pembeli di swedia ada 36 orang dan jumlah item/barang ada 261

In [ ]:
# encoding
def hot_encode(x):
  if(x <=0):
    return 0
  if(x >=1):
    return 1

In [ ]:
# encode kan ke tiap tiap wilayah

basket_encoded = basket_france.applymap(hot_encode)
basket_france = basket_encoded

basket_encoded = basket_portugal.applymap(hot_encode)
basket_portugal = basket_encoded

basket_encoded = basket_swedia.applymap(hot_encode)
basket_swedia = basket_encoded

In [ ]:
basket_swedia

Description,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE WOODLAND,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 RAFFIA RIBBONS VINTAGE CHRISTMAS,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,4 PEAR BOTANICAL DINNER CANDLES,4 VANILLA BOTANICAL CANDLES,5 HOOK HANGER MAGIC TOADSTOOL,6 GIFT TAGS 50'S CHRISTMAS,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,60 CAKE CASES VINTAGE CHRISTMAS,60 TEATIME FAIRY CAKE CASES,72 SWEETHEART FAIRY CAKE CASES,ABC TREASURE BOOK BOX,ADVENT CALENDAR GINGHAM SACK,AIRLINE BAG VINTAGE TOKYO 78,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOURS SILK FAN,BAG 250g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING MOULD HEART MILK CHOCOLATE,BAKING MOULD HEART WHITE CHOCOLATE,BAKING MOULD ROSE MILK CHOCOLATE,BAKING MOULD ROSE WHITE CHOCOLATE,BAKING MOULD TOFFEE CUP CHOCOLATE,BINGO SET,BLACK KITCHEN SCALES,BLACK LOVE BIRD CANDLE,BLUE HAPPY BIRTHDAY BUNTING,BLUE POLKADOT GARDEN PARASOL,BLUE POLKADOT WRAP,BLUE VINTAGE SPOT BEAKER,BOTTLE BAG RETROSPOT,BOX OF 6 CHRISTMAS CAKE DECORATIONS,BOX OF 6 MINI 50'S CRACKERS,BOX OF 6 MINI VINTAGE CRACKERS,BOYS VINTAGE TIN SEASIDE BUCKET,BUBBLEGUM RING ASSORTED,BUFFALO BILL TREASURE BOOK BOX,BUNDLE OF 3 RETRO NOTE BOOKS,CARAVAN SQUARE TISSUE BOX,CHARLOTTE BAG APPLES DESIGN,CHARLOTTE BAG PINK POLKADOT,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY DOLLY GIRL,CHILDRENS CUTLERY POLKADOT PINK,CHILDRENS TOY COOKING UTENSIL SET,CHRISTMAS METAL POSTCARD WITH BELLS,CINNAMON SCENTED VOTIVE CANDLE,CUPCAKE LACE PAPER SET 6,DINOSAUR PARTY BAG + STICKER SET,DOILEY STORAGE TIN,EAU DE NIL LOVE BIRD CANDLE,FAIRY CAKE FLANNEL ASSORTED COLOUR,FAIRY TALE COTTAGE NIGHTLIGHT,"FEATHER PEN,HOT PINK","FEATHER PEN,LIGHT PINK",FELT FARM ANIMAL WHITE BUNNY,FELT TOADSTOOL SMALL,FELTCRAFT DOLL ROSIE,FELTCRAFT PRINCESS CHARLOTTE DOLL,FIRST AID TIN,GENTLEMAN SHIRT REPAIR KIT,GIFT BAG BIRTHDAY,GIRLS VINTAGE TIN SEASIDE BUCKET,GRAND CHOCOLATECANDLE,GREEN METAL BOX ARMY SUPPLIES,GREEN VINTAGE SPOT BEAKER,GUMBALL COAT RACK,GYMKHANA TREASURE BOOK BOX,HAIRCLIPS FORTIES FABRIC ASSORTED,HAND WARMER BIRD DESIGN,HAND WARMER OWL DESIGN,HAND WARMER UNION JACK,HAPPY STENCIL CRAFT,HEART STRING MEMO HOLDER HANGING,HOMEMADE JAM SCENTED CANDLES,HYACINTH BULB T-LIGHT CANDLES,ICE CREAM BUBBLES,ICE CREAM SUNDAE LIP GLOSS,IVORY LOVE BIRD CANDLE,JAM JAR WITH GREEN LID,JAM JAR WITH PINK LID,JAM MAKING SET PRINTED,JAM MAKING SET WITH JARS,JIGSAW TOADSTOOLS 3 PIECE,JUMBO BAG ALPHABET,JUMBO BAG APPLES,JUMBO BAG OWLS,JUMBO BAG RED RETROSPOT,JUMBO BAG WOODLAND ANIMALS,KIDS RAIN MAC BLUE,KIDS RAIN MAC PINK,KINGS CHOICE TEA CADDY,LARGE PURPLE BABUSHKA NOTEBOOK,LARGE RED BABUSHKA NOTEBOOK,LARGE YELLOW BABUSHKA NOTEBOOK,LIPSTICK PEN BABY PINK,LUNCH BAG APPLE DESIGN,LUNCH BAG CARS BLUE,LUNCH BAG PINK POLKADOT,LUNCH BAG RED RETROSPOT,LUNCH BAG WOODLAND,LUNCH BOX WITH CUTLERY RETROSPOT,MAGIC DRAWING SLATE CIRCUS PARADE,MAGIC DRAWING SLATE DOLLY GIRL,MAGIC DRAWING SLATE PURDEY,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW DOLLY GIRL,MINI JIGSAW PURDEY,MINI JIGSAW SPACEBOY,MINI LIGHTS WOODLAND MUSHROOMS,MINI PAINT SET VINTAGE,MINI PLAYING CARDS BUFFALO BILL,MINI PLAYING CARDS DOLLY GIRL,MINI PLAYING CARDS GYMKHANA,MINI PLAYING CARDS SPACEBOY,MOBILE VINTAGE HEARTS,MONSTERS STENCIL CRAFT,MR ROBOT SOFT TOY,MULTICOLOUR HONEYCOMB PAPER GARLAND,Manual,NOEL GARLAND PAINTED ZINC,ORANGE VOTIVE CANDLE,PACK OF 12 COLOURED PENCILS,PACK OF 12 TRADITIONAL CRAYONS,PACK OF 12 WOODLAND TISSUES,PACK OF 20 NAPKINS PANTRY DESIGN,PACK OF 60 DINOSAUR CAKE CASES,PACK OF 60 PINK PAISLEY CAKE CASES,PACK OF 60 SPACEBOY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,PACK OF 72 SKULL CAKE CASES,PAIR PADDED HANGERS PINK CHECK,PANTRY PASTRY BRUSH,PANTRY SCRUBBING BRUSH,PANTRY WASHING UP BRUSH,PAPER BUNTING PAISLEY PARK,PAPER BUNTING RETROSPOT,PAPER CHAIN KIT RETR

# membangun model

In [ ]:
# membangun model dan menganalisis hasilnya

# 1. prancis

# minimal suportnya 0.05
frq_items = apriori(basket_france, min_support= 0.05, use_colnames = True)

# mengumpulkan aturan yang disimpu;kan dalam dataframe
rules = association_rules(frq_items, metric='lift', min_threshold =1)
rules = rules.sort_values(['confidence','lift'], ascending= [False, False])
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
45,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000,1.306667,0.017961,inf,0.254144
260,"(PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...",(POSTAGE),0.051020,0.765306,0.051020,1.000,1.306667,0.011974,inf,0.247312
271,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...",(POSTAGE),0.053571,0.765306,0.053571,1.000,1.306667,0.012573,inf,0.247978
302,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975,7.644000,0.086474,34.897959,0.967949
300,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975,7.077778,0.085433,34.489796,0.956294


antecedence itu anggap saja A
consequents itu anggap saja B
jadi jika orang membeli jumbo bag woodland animals maka ia membeli postage

In [ ]:
# portugal

frq_items = apriori(basket_portugal, min_support= 0.05, use_colnames = True)

# mengumpulkan aturan yang disimpu;kan dalam dataframe
rules = association_rules(frq_items, metric='lift', min_threshold =1)
rules = rules.sort_values(['confidence','lift'], ascending= [False, False])
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1170,(SET 12 COLOUR PENCILS DOLLY GIRL),(SET 12 COLOUR PENCILS SPACEBOY),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf,1.0
1171,(SET 12 COLOUR PENCILS SPACEBOY),(SET 12 COLOUR PENCILS DOLLY GIRL),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf,1.0
1172,(SET 12 COLOUR PENCILS DOLLY GIRL),(SET OF 4 KNICK KNACK TINS LONDON),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf,1.0
1173,(SET OF 4 KNICK KNACK TINS LONDON),(SET 12 COLOUR PENCILS DOLLY GIRL),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf,1.0
1174,(SET 12 COLOUR PENCILS DOLLY GIRL),(SET OF 4 KNICK KNACK TINS POPPIES),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf,1.0


## insight
- antecedents: Ini adalah kumpulan item yang ada sebelum hasil yang diinginkan dalam aturan asosiasi. Dalam contoh yang Anda berikan, antecedents adalah '(SET 12 COLOUR PENCILS DOLLY GIRL)'.

- consequents: Ini adalah item atau kumpulan item yang dihasilkan setelah item atau kumpulan item dalam antecedents dalam aturan asosiasi. Dalam contoh yang Anda berikan, consequents adalah '(SET 12 COLOUR PENCILS SPACEBOY)'.

- antecedent support: Ini adalah proporsi transaksi yang mengandung antecedents dalam keseluruhan dataset. Dalam contoh yang Anda berikan, antecedent support adalah 0.051724, yang berarti sekitar 5.17% dari semua transaksi mengandung '(SET 12 COLOUR PENCILS DOLLY GIRL)'.

- consequent support: Ini adalah proporsi transaksi yang mengandung consequents dalam keseluruhan dataset. Dalam contoh yang Anda berikan, consequent support adalah 0.051724, yang berarti sekitar 5.17% dari semua transaksi mengandung '(SET 12 COLOUR PENCILS SPACEBOY)'.

- support: Ini adalah proporsi transaksi yang mengandung baik antecedents maupun consequents dalam keseluruhan dataset. Dalam contoh yang Anda berikan, support adalah 0.051724, yang berarti sekitar 5.17% dari semua transaksi mengandung kedua item tersebut.

- confidence: Ini adalah seberapa sering aturan ini ditemukan benar. Dalam contoh yang Anda berikan, confidence adalah 1.0, yang berarti setiap kali '(SET 12 COLOUR PENCILS DOLLY GIRL)' dibeli, '(SET 12 COLOUR PENCILS SPACEBOY)' juga dibeli.

In [ ]:
# swedia

frq_items = apriori(basket_swedia, min_support= 0.05, use_colnames = True)

# mengumpulkan aturan yang disimpu;kan dalam dataframe
rules = association_rules(frq_items, metric='lift', min_threshold =1)
rules = rules.sort_values(['confidence','lift'], ascending= [False, False])
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
25485,(POSTAGE),"(CHILDRENS CUTLERY DOLLY GIRL, CHILDRENS CUTLE...",0.611111,0.055556,0.055556,0.090909,1.636364,0.021605,1.038889,1.000000
202,(POSTAGE),(CUPCAKE LACE PAPER SET 6),0.611111,0.083333,0.055556,0.090909,1.090909,0.004630,1.008333,0.214286
385,(POSTAGE),(MINI PLAYING CARDS DOLLY GIRL),0.611111,0.083333,0.055556,0.090909,1.090909,0.004630,1.008333,0.214286
482,(POSTAGE),(ROUND SNACK BOXES SET OF4 WOODLAND),0.611111,0.083333,0.055556,0.090909,1.090909,0.004630,1.008333,0.214286
3571,(POSTAGE),"(ROUND SNACK BOXES SET OF4 WOODLAND, ROUND SNA...",0.611111,0.083333,0.055556,0.090909,1.090909,0.004630,1.008333,0.214286


In [ ]:
rules.shape

(104608, 10)